### Nous présentons ici la solution finale de classification pour la deuxième phase du stage après plusieurs expérimentations 

### 1. Les données

Nous avons formé les données en utilisant les sources de padi-web (VS en SV), ESV, ESA ainsi que les articles nettoyés de la phase 1 du stage (ceux dans lequels on avait supprimé les noms de maladie, les liens hypertextes, détecté la langue)

### 2. Prétaitement 

Sur ces données collectées et rassemblées dans le fichier *data_classification_2.csv*, nous avons appliqué un nettoyage des textes pour former la colonne *text_clean*, que nous utilisons pour la classification. Nous allons appliqué la suppression des stopword, la lemmatisation, la suppression des caractères incompréhensibles en anglais, la mise en minuscule 

### 3. Classification 

Nous avons testé plusieurs approches à l'instar de: 
- Vectorisation avec TF-IDF et puis classification avec Naive Bayes 
- Vectorisation avec TF-IDF et puis classification avec la Regression Logistique 
- Vectorisation avec SBERT et puis classification avec la Regression Logistique
- Vectorisation avec SBERT et puis classification avec MLP de pytorch (ainsi que MLPClassifier de scikit-learn)

